In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/qa-intents-dataset-university-domain/dataset_test.tsv
/kaggle/input/qa-intents-dataset-university-domain/labels_description.txt
/kaggle/input/qa-intents-dataset-university-domain/dataset_train.tsv


In [11]:
train_data = pd.read_csv('../input/qa-intents-dataset-university-domain/dataset_train.tsv',delimiter='\t',encoding="utf-8",names=['text', 'intent'])
test_data = pd.read_csv('../input/qa-intents-dataset-university-domain/dataset_test.tsv',delimiter='\t',encoding="utf-8",names=['text', 'intent'])
train_data.head()

,text,intent
0,мне нужна справка,statement_general
1,оформить справку,statement_general
2,взять справку,statement_general
3,справку как получить,statement_general
4,справку ммф где получаться,statement_general


In [12]:
full_data = pd.concat([train_data, test_data])
full_data.head()

,text,intent
0,мне нужна справка,statement_general
1,оформить справку,statement_general
2,взять справку,statement_general
3,справку как получить,statement_general
4,справку ммф где получаться,statement_general


In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(full_data, test_size=0.2, random_state=42)
unique_values_normalized = train['intent'].value_counts(normalize=True)
print(unique_values_normalized*100)

intent
sched_teacher                8.582817
sched_for_group              3.126661
sched_for_group_day          3.046944
wifi                         2.143490
status_free                  2.037201
                               ...   
location_general             0.168291
loc_nsu_cafeteria            0.168291
student_trade_union_enter    0.159433
loc_passport_office          0.159433
staff_trade_union_enter      0.159433
Name: proportion, Length: 142, dtype: float64


In [14]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_recall_fscore_support, balanced_accuracy_score
from sklearn import svm
import timeit

In [17]:
# Создание пайплайна
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=lambda x: tokenizer.encode(x, add_special_tokens=True))),
    ('svm', svm.SVC())  
])

# Обучение пайплайна
pipeline.fit(train['text'], train['intent'])

start_test = timeit.default_timer()
# Прогноз на тестовом наборе
predictions = pipeline.predict(test['text'])
end_test = timeit.default_timer()
# Общее количество предсказанных ответов
num_predictions = len(predictions)

# Среднее время на один ответ
average_time_per_response = (end_test - start_test) / num_predictions

print(f'Time for testing: {end_test - start_test:.4f} seconds')
print(f'Average time per response: {average_time_per_response:.6f} seconds')
print(balanced_accuracy_score(test['intent'], predictions))
print(precision_recall_fscore_support(test['intent'], predictions, average = 'weighted'))

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Time for testing: 13.5083 seconds
Average time per response: 0.004785 seconds
0.9413793172812713
(0.9426154953864635, 0.9440311725115126, 0.941858187955414, None)


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(predictions[0])

In [ ]:
# Оценка точности
# acc = accuracy_score(test_data['intent'], predictions)
# rec = recall_score(test_data['intent'], predictions, average='weighted')
# f1 = f1_score(test_data['intent'], predictions, average='weighted')
# print(f"Accuracy: {acc:.4f}, Recall: {rec:.4f}, F1-score: {f1:.4f}")

In [18]:
# Сохранение входных данных, истинных меток и предсказаний в DataFrame
results_df = pd.DataFrame({
    'Input': test['text'],
    'True Label': test['intent'],
    'Predicted Label': predictions
})

# Сохранение DataFrame в CSV файл
#results_df.to_csv('model_predictions.csv', index=False)


In [19]:
# Вывод первых нескольких строк для проверки
print(results_df.head(12))

                                                 Input  \
8079              распологается где расписание занятий   
4950                      усваивать семинарист сколько   
9941                            где есть экзамен число   
600   как мне заказать разрешение на внос оборудования   
1121                               где есть деканат эф   
3821                                       комиссариат   
6595                        где есть основа бюджетный.   
2935                       новое здание НГУ пожалуйста   
5762                               абонентнсунетстать!   
2342                  институт электрометрии раскопать   
7346                               атм расположен где)   
9615                    получаться wifi в университете   

                   True Label          Predicted Label  
8079           sched_location           sched_location  
4950   stat_numb_of_academics   stat_numb_of_academics  
9941               sched_exam               sched_exam  
600   student_uni

In [20]:
print(results_df[results_df['True Label']!=results_df['Predicted Label']])

                                                  Input  \
2662                      пройти институт электрометрии   
5597                                   неважно обедать?   
2150  институт автоматики и электрометрии со ран рас...   
2444              расположен где институт электрометрии   
2495                  институт электрометрии обнаружить   
...                                                 ...   
2218                   институт электрометрии находится   
2238  институт автоматики и электрометрии со ран нах...   
2089              распологается где институт автоматики   
175   как пройти институт автоматики и электрометрии...   
4250                        я хочу видеть план корпусов   

                               True Label         Predicted Label  
2662               loc_termophysinstitute  loc_philologyinstitute  
5597                              loc_eat                loc_shop  
2150  loc_automationelectrometryinstitute       loc_chiminstitute  
2444               

mini-LM

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, balanced_accuracy_score

In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Load your dataset
train_data = pd.read_csv('../input/qa-intents-dataset-university-domain/dataset_train.tsv', delimiter='\t', encoding="utf-8", names=['text', 'intent'])
test_data = pd.read_csv('../input/qa-intents-dataset-university-domain/dataset_test.tsv', delimiter='\t', encoding="utf-8", names=['text', 'intent'])
full_data = pd.concat([train_data, test_data])

# Split the data into train and test sets
train, test = train_test_split(full_data, test_size=0.2, random_state=42)

# Tokenize and encode the text data
train_encodings = tokenizer(train['text'].tolist(), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(test['text'].tolist(), truncation=True, padding=True, return_tensors="pt")

In [ ]:
# Extract embeddings
with torch.no_grad():
    train_embeddings = model(**train_encodings).pooler_output
    test_embeddings = model(**test_encodings).pooler_output

In [ ]:
svm_ = svm.SVC()

svm_.fit(train_embeddings, train['intent'])

test_embeddings_array = test_embeddings.numpy()


start_test = timeit.default_timer()
# Прогноз на тестовом наборе
predictions = svm_.predict(test_embeddings_array)
#predictions = pipeline.predict(test_data['text'])
end_test = timeit.default_timer()
# Общее количество предсказанных ответов
num_predictions = len(predictions)

# Среднее время на один ответ
average_time_per_response = (end_test - start_test) / num_predictions

print(f'Time for testing: {end_test - start_test:.4f} seconds')
print(f'Average time per response: {average_time_per_response:.6f} seconds')


In [ ]:
print("precision_recall_fscore weighted", precision_recall_fscore_support(test['intent'], predictions, average='weighted'))
print("balanced_accuracy", balanced_accuracy_score(test['intent'], predictions))

M-USE

In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("sadakmed/distiluse-base-multilingual-cased-v2")
model = AutoModel.from_pretrained("sadakmed/distiluse-base-multilingual-cased-v2")

# Load your dataset
train_data = pd.read_csv('../input/qa-intents-dataset-university-domain/dataset_train.tsv', delimiter='\t', encoding="utf-8", names=['text', 'intent'])
test_data = pd.read_csv('../input/qa-intents-dataset-university-domain/dataset_test.tsv', delimiter='\t', encoding="utf-8", names=['text', 'intent'])
full_data = pd.concat([train_data, test_data])

# Split the data into train and test sets
train, test = train_test_split(full_data, test_size=0.2, random_state=42)

# Tokenize and encode the text data
train_encodings = tokenizer(train['text'].tolist(), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(test['text'].tolist(), truncation=True, padding=True, return_tensors="pt")

In [ ]:
# Извлечение эмбеддингов
with torch.no_grad():
    train_embeddings = model(**train_encodings).last_hidden_state.mean(dim=1)
    test_embeddings = model(**test_encodings).last_hidden_state.mean(dim=1)

In [ ]:
svm_ = svm.SVC()

svm_.fit(train_embeddings, train['intent'])

test_embeddings_array = test_embeddings.numpy()


start_test = timeit.default_timer()
# Прогноз на тестовом наборе
#predictions = pipeline.predict(test_data['text'])
predictions = svm_.predict(test_embeddings_array)
end_test = timeit.default_timer()
# Общее количество предсказанных ответов
num_predictions = len(predictions)

# Среднее время на один ответ
average_time_per_response = (end_test - start_test) / num_predictions

print(f'Time for testing: {end_test - start_test:.4f} seconds')
print(f'Average time per response: {average_time_per_response:.6f} seconds')

In [ ]:
print("precision_recall_fscore weighted", precision_recall_fscore_support(test['intent'], predictions, average='weighted'))
print("balanced_accuracy", balanced_accuracy_score(test['intent'], predictions))